In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install OpenAI
import json
import pandas as pd
import numpy as np
from ParserDB import ParserDB

In [3]:
def stances_by_thread(data):
    user_stances = {} # {user: {supports: 3, ...}}
    
    with open(data, "r") as f:
        for l in f.readlines():
            response_text = json.loads(json.loads(l)["response"]["body"]["output"][0]["content"][0]["text"])
            for thread in response_text["threads"]:
                for user in thread["users"]:
                    username = user["user"]
                    userstance = user["stance"]
                    cur_user_stance = user_stances.get(username, {})
                    cur_user_stance[userstance] = cur_user_stance.get(userstance, 0) + 1
                    user_stances[username] = cur_user_stance
    return user_stances

def define_user_stance(stance_counts):
    if stance_counts.get("поддерживает", 0) == stance_counts.get("не поддерживает", 0):
        return "невозможно определить"
    return "поддерживает" if stance_counts.get("поддерживает", 0) > stance_counts.get("не поддерживает", 0) else "не поддерживает"

def make_graph(threads_by_videos, user_stances, folder_to_save):
    nodes = set()
    edges = set()
    
    reply_count = 0
    problem_count = 0
    for videoId, videoThreads, query in threads_by_videos:
        videoThreads = json.loads(videoThreads)
        users = set()
        thread_authors = dict() # {topLevelComment: author}
        nodes.add((videoId, "video", query))
        for thread in videoThreads:
            thread = json.loads(thread)
            for comment in thread:
                users.add(comment["authorDisplayName"])
                nodes.add((comment["authorDisplayName"], "user", "USER"))
                if comment["topLevelComment"] == comment["commentId"]:
                    thread_authors[comment["topLevelComment"]] = comment["authorDisplayName"]
                
        for thread in videoThreads:
            thread = json.loads(thread)
            thread_author = thread_authors[thread[0]["topLevelComment"]]
            for comment in thread:
                if comment["authorDisplayName"] == thread_author:
                    edges.add((comment["authorDisplayName"], videoId))
                elif "@" not in comment["text"]:
                    edges.add((comment["authorDisplayName"], thread_author))
                else:
                    reply_to = None
                    for user in users:
                        if user in comment["text"]:
                            reply_to = user
                            break
                    if reply_to is not None:
                        edges.add((comment["authorDisplayName"], reply_to))
                    else:
                        problem_count += 1

    nodes = [dict(id=n[0], label=n[0], nodeType=n[1], query=n[2], stance=define_user_stance(user_stances.get(n[0], {}))) for n in nodes]
    edges = [dict(source=e[0], target=e[1]) for e in edges]
    
    nodes_df = pd.DataFrame(nodes)
    nodes_df["size"] = np.where(nodes_df["nodeType"] == "video", 20, 1)
    nodes_df.to_csv(f"{folder_to_save}/nodes.csv", index=False, encoding="utf-8")
    pd.DataFrame(edges).to_csv(f"{folder_to_save}/edges.csv", index=False, encoding="utf-8")

In [4]:
p = ParserDB(api_key_file="./data/data_json/api_token.txt", database="./data/data.db")
chemtrails_threads = p.get_threads("химтрейлы")
chemtrails_stances = stances_by_thread("./data/openai_responses/batch_chemtrails_output.jsonl")

In [5]:
make_graph(chemtrails_threads, chemtrails_stances, "data/gephi_chem_stances")